In [1]:
import pandas as pd
import os
%matplotlib inline

In [2]:
import seaborn

In [3]:
print(os.getcwd())
os.listdir()

/Users/ah-bau/github_repo/electricity_data_nctu/02code


['.ipynb_checkpoints',
 '01_資料欄位前處理.ipynb',
 'project2018-dorm09.ipynb',
 'project2018-dorm10-graduate01.ipynb',
 'project2018-dorm10.ipynb',
 'project2018-dorm_87.ipynb',
 'project2018-dorm_chu.ipynb',
 'project2018_main-now.ipynb',
 'project2018_main.ipynb',
 'project2018_v0513.ipynb']

In [4]:
# Global variables
CostRate = 2.8 # 一度電多少錢（估算，實際還要再確認）

In [8]:
file = 'D12_hr_20170831_20180228.csv'
path = '../01data/dorm/{}'.format(file)
df = pd.read_csv(path)

In [9]:
print('Old names: ',df.columns)
print('Reassign Column names')
df.columns = ['DateTime', 'Area', 'Building', 'Loop', 'Type', 'Usage']
print('New names: ',df.columns)
df.head()

Old names:  Index(['時  間', '區 域', '館 舍 別', '迴 路 別', '用電別', '用電量(單位:度)'], dtype='object')
Reassign Column names
New names:  Index(['DateTime', 'Area', 'Building', 'Loop', 'Type', 'Usage'], dtype='object')


,DateTime,Area,Building,Loop,Type,Usage
0,2017/08/31 00:00,南區,學生十二舍,主迴路,宿舍,33.5
1,2017/08/31 01:00,南區,學生十二舍,主迴路,宿舍,34.0
2,2017/08/31 02:00,南區,學生十二舍,主迴路,宿舍,33.5
3,2017/08/31 03:00,南區,學生十二舍,主迴路,宿舍,36.5
4,2017/08/31 04:00,南區,學生十二舍,主迴路,宿舍,35.0


In [10]:
def clear_white_space(df):
    print("Before: ", df.Building.unique())
    for y in df.select_dtypes(include = 'object').columns:
        df[y] = df[y].str.strip()
    print('After: ', df.Building.unique())
clear_white_space(df)

Before:  ['學生十二舍               ']
After:  ['學生十二舍']


In [11]:
# 用自訂函式去理解DF中每個欄位有哪幾種類別的值
print_limit = 100
def understand(df):
    for col_name in df.columns:
        print ('>>> "{}" has {} unique values:'.format(col_name, len(df[col_name].unique())))
        if(len(df[col_name].unique())< print_limit):
            print(df[col_name].unique())
        else:
            print('The value is more than {} so wont be list here :P'.format(print_limit))
        print('\n')


In [12]:
understand(df)

>>> "DateTime" has 4361 unique values:
The value is more than 100 so wont be list here :P


>>> "Area" has 1 unique values:
['南區']


>>> "Building" has 1 unique values:
['學生十二舍']


>>> "Loop" has 1 unique values:
['主迴路']


>>> "Type" has 1 unique values:
['宿舍']


>>> "Usage" has 368 unique values:
The value is more than 100 so wont be list here :P




### 原始資料說明 - 2
- index: ['﻿時  間', '區 域', '館 舍 別', '迴 路 別', '用電別', '用電量(單位:度)']

### 人工資料前處理：改變欄位名稱
#### 改變的原因
- 原始的資料有空白在字串裡，會造成一些麻煩
- 英文的名稱，可以在Pandas寫Code時較為方便(才能用 df.Usage的指令)

#### 改了什麼
- 我用Number(Excel)的尋找取代功能，把資料轉成英文，以下括弧表示原始資料的命名
    - 區域：southArea(南區), northArea(北區)
    - 迴 路 別: mainLoop（主迴路）, familyMart（1F便利超商）, club（社團）, socialHall（閱覽交誼廳）, studentHall（學生交誼廳）, streetLight(環校路燈),
    - 館舍： dorm_13（學生十三舍）,(7,8,9,10,11,12 以此類推), dorm_graduate_2（研二）, dorm_87（學生七舍＿八舍）, dorm_13_graduate_1(11與研一), dorm_chu（竹軒）
    - 用電別：rent（外租）, dorm（宿舍）, admin(行政)

In [13]:
dic_replace = {
    'Area' : {'北區':'N','南區':'S'},
    'Building' : {
        '基礎大樓':"SB3",
        '科學二館':"SB2",
        '科學一館':"SB1",
        '學生十三舍': "D13", 
        '學生十二舍': 'D12',
        '學生十一舍': 'D11',
        '學生十舍': 'D10',
        '學生九舍': 'D9',
        '學生八舍': 'D8',
        '學生七舍': 'D7',
        '學生七舍_八舍':'D87',
        '竹軒宿舍': 'Df1',
        '女二舍': 'Df2',
        '研究生一舍': 'Dgd1',
        '研究生二舍': 'Dgd2',
        '學生十一舍_研一舍':'D11_grad1',
        '管理一館':'MB1',
        '管理二館':'MB2',
        '工程一館': 'EB1',
        '工程二館': 'EB2',
        '工程三館': 'EB3',
        '工程四館': 'EB4',
        '工程五館': 'EB5',
        '工程六館': 'EB6',
        '人社一館': 'HS1',
        '人社二館': 'HS2',
        '交映樓': 'CPT',
        '土木結構': 'CElab',
        '中正堂':'CKS',
        '光電大樓':'TKB',
        '第二餐廳': 'Canteen2',
        '服務大樓':'Service',
        '行政大樓':'Administration',
        '環保大樓':'Environmental',
        '材料實驗':'Material',
        '綜合一館':'Assembly1',
        '綜合球館':'Sport',
        '游泳館':'Swim',
        '體育館':'Gym',
        '機車Ｈ棚':'ShedH',
        '機車Ａ棚':'ShedA',
        '機車Ｄ棚':'ShedD',
        '工程一館中繼':'PowerSatatiopn_EB1',
        '人社二館中繼':'PowerSatatiopn_HS2',
        '南區總電站':'PowerSatatiopn_main_south',
        '防災中心':'Safty',
        '污水場':'Wastewater',
        '電資大樓': 'ElecInfo',
        '資訊館(南區)':'Info_south',
        '資訊館(北區)':'Info_north',
        '活動中心': 'Activity',
        '圖資中心': 'Library',
        '第一招待所': 'GuestHouse'
                 },
    'Type' : {'教學': 'edu', '宿舍': 'dorm', '行政':'admin', '外租':'rent','電站':'power'},
    'Loop' : {'主迴路': 'main'}
              }
# ,'Building': {'學生十舍':'d10', '學生十三舍':'d13', }
df = df.replace(dic_replace)

In [14]:
df.Building.unique()

array(['D12'], dtype=object)

In [15]:
understand(df)

>>> "DateTime" has 4361 unique values:
The value is more than 100 so wont be list here :P


>>> "Area" has 1 unique values:
['S']


>>> "Building" has 1 unique values:
['D12']


>>> "Loop" has 1 unique values:
['main']


>>> "Type" has 1 unique values:
['dorm']


>>> "Usage" has 368 unique values:
The value is more than 100 so wont be list here :P




In [16]:
os.chdir(path[:path.rfind('/')])

In [17]:
def generate_date_time_columns(df):
    if('DateTime' in df.columns):
            df['Date'] = df.DateTime.str[:10]
            df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
            df['Weekday'] = df['Date'].dt.dayofweek + 1
            df['Time'] = df.DateTime.str[11:]
            df['Week'] = df['Date'].dt.week
            print('Columns #Date, #Time, #Weekday, #Week has been generated from DateTime and added to DataFrame')
    else:
        print('DateTime not in column. Please rename columns name.')
    return df

In [18]:
# cost_rate：一度電大概多少錢
def generate_cost_columns(df, cost_rate):
    df['Cost({})'.format(CostRate)] = df.Usage * cost_rate
    print('Column "Cost" had been generated!')
    return df

In [19]:
df = generate_date_time_columns(df)
df = generate_cost_columns(df, CostRate)
df.head()

Columns #Date, #Time, #Weekday, #Week has been generated from DateTime and added to DataFrame
Column "Cost" had been generated!


,DateTime,Area,Building,Loop,Type,Usage,Date,Weekday,Time,Week,Cost(2.8)
0,2017/08/31 00:00,S,D12,main,dorm,33.5,2017-08-31,4,00:00,35,93.8
1,2017/08/31 01:00,S,D12,main,dorm,34.0,2017-08-31,4,01:00,35,95.2
2,2017/08/31 02:00,S,D12,main,dorm,33.5,2017-08-31,4,02:00,35,93.8
3,2017/08/31 03:00,S,D12,main,dorm,36.5,2017-08-31,4,03:00,35,102.2
4,2017/08/31 04:00,S,D12,main,dorm,35.0,2017-08-31,4,04:00,35,98.0


In [20]:
os.getcwd()

'/Users/ah-bau/github_repo/electricity_data_nctu/01data/dorm'

In [21]:
df.to_csv('[new]'+file, encoding='utf-8')
# 存檔，作業完成！